# Redes Neurais Artificiais 2020.1 -- Projeto Prático 3.4

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

In [15]:
# Começando com os imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pickle
import warnings

# Isso é para ignorar os warnings dizendo que a rede atingiu o numero maximo de iterações e não convergiu
warnings.filterwarnings('ignore') 

In [16]:
# Carregando a base...
df = pd.read_csv('../data/covtype.csv')
df.head()

# Separando os atributos preditores do atributo alvo
x = df.drop(['Cover_Type'],axis = 1)
y = df['Cover_Type']

In [17]:
# Top 6 arquiteturas (3 próprias seguidas de 3 com regra da pirâmide)

#   (hidden_layers, activation, solver, epochs, uses_cat, repetitions)
top = [
    (10, 'tanh', 'adam', 150, False, 10),
    (10, 'tanh', 'adam', 200, False, 10),
    (10, 'tanh', 'adam', 200, True, 5)
]
top_pyramid = [
    ((13, 13), 'tanh', 'adam', 200, True, 5),
    ((16, 10), 'tanh', 'adam', 200, True, 5),
    ((26), 'tanh', 'adam', 200, True, 5)
]

In [18]:
# Função de apoio para interpretação dos dados salvos de métricas
def load_top_metrics(path, n, array_acc, array_fsc, load_as_results = True):
    f = open(path, "rb")
    temp = pickle.load(f)
    f.close()

    if load_as_results:
        for a,r,p,f in temp[:n]:
            array_acc.append(np.mean(a))
            array_fsc.append(np.mean(f))
        return
    
    top_acc = np.argpartition(np.array(list(temp.values()))[:, 0], -n)[-n:]
    top_fscr = np.argpartition(np.array(list(temp.values()))[:, 1], -n)[-n:]

    top = np.array(list(temp.keys()))[top_fscr]

    for c in top:
        if not isinstance(c, int):
            c = tuple(c)
        array_acc.append(temp[c][0])
        array_fsc.append(temp[c][1])

    return top

In [19]:
# Carrega as métricas do top 6
metrics_acc = []
metrics_fsc = []
pyramid_acc = []
pyramid_fsc = []

load_top_metrics("../metrics_3.3/metrics_dict.pkl", 2, metrics_acc, metrics_fsc, load_as_results = False)
load_top_metrics("../metrics_3.3/metrics_catg_dict.pkl", 1, metrics_acc, metrics_fsc)
load_top_metrics("../metrics_3.3/piramide_metrics_catg_dict.pkl", 3, pyramid_acc, pyramid_fsc)

In [20]:
print("Top 3 arquiteturas próprias")
print("------------------------------------------------------------------------------------------------------")
print("(hidden_layers, activation, solver, epochs, usa categóricos, repetitions) -> F-Score, Acurácia (média)")
print("------------------------------------------------------------------------------------------------------")

for c,f,a in zip(top, metrics_fsc, metrics_acc):
    print("{} -> {:.2f} ; {:.2f}".format(c, f, a))

print("\n\nTop 3 arquiteturas com regra da pirâmide")
print("------------------------------------------------------------------------------------------------------")
print("(hidden_layers, activation, solver, epochs, usa categóricos, repetitions) -> F-Score, Acurácia (média)")
print("------------------------------------------------------------------------------------------------------")

for c,f,a in zip(top_pyramid, pyramid_fsc, pyramid_acc):
    print("{} -> {:.2f} ; {:.2f}".format(c, f, a))

Top 3 arquiteturas próprias
------------------------------------------------------------------------------------------------------
(hidden_layers, activation, solver, epochs, usa categóricos, repetitions) -> F-Score, Acurácia (média)
------------------------------------------------------------------------------------------------------
(10, 'tanh', 'adam', 150, False, 10) -> 0.51 ; 0.72
(10, 'tanh', 'adam', 200, False, 10) -> 0.53 ; 0.72
(10, 'tanh', 'adam', 200, True, 5) -> 0.59 ; 0.77


Top 3 arquiteturas com regra da pirâmide
------------------------------------------------------------------------------------------------------
(hidden_layers, activation, solver, epochs, usa categóricos, repetitions) -> F-Score, Acurácia (média)
------------------------------------------------------------------------------------------------------
((13, 13), 'tanh', 'adam', 200, True, 5) -> 0.68 ; 0.80
((16, 10), 'tanh', 'adam', 200, True, 5) -> 0.68 ; 0.81
(26, 'tanh', 'adam', 200, True, 5) -> 0.71 ; 

In [23]:
# Define os parâmetros do Grid Search

searchers = []
for layers, activ, solver, epochs, cat, reps in top + top_pyramid:
    # Necessário para executar a padronização a cada parte do treinamento
    pipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', MLPClassifier(hidden_layer_sizes = layers, activation = activ))
    ])
    parameters = {
        'solver': ('adam', 'sgd'),
        'batch_size': ['auto'],
        'learning_rate_init': [0.001, 0.01, 0.1, 1.0],
        'n_iter_no_change': [10, 30, 50, 100], # paciência
        'max_iter': [100, 150, 200, 250] # épocas
    }
    searchers.append(GridSearchCV(pipeline, parameters)) # validação 5-Fold é padrão

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html
2. Treine o modelo com todos os dados
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas